In [27]:
import math
x = [1, 2]
y = [1, -6]


math.sqrt((x[0] - y[0])**2 + (x[1] - y[1])**2) # Euclidean distance

8.0

In [28]:
(x[0]*y[0] + x[1]*y[1]) / (4*6)   # Dot product

-0.4583333333333333

In [29]:
import torch 

In [30]:
sentence = "this is a sample sentence for embedding"
sentence2 = "this is sentence embedding"

dc = {s:i for i,s in enumerate(sorted(sentence.replace(',', '').split()))}
dc

{'a': 0,
 'embedding': 1,
 'for': 2,
 'is': 3,
 'sample': 4,
 'sentence': 5,
 'this': 6}

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(max_features=10,binary=True)
cv.fit([sentence])
cv.transform([sentence]).todense()

matrix([[1, 1, 1, 1, 1, 1]])

In [32]:
cv.transform([sentence2]).todense()
cv.transform(["this"]).todense()

matrix([[0, 0, 0, 0, 0, 1]])

In [33]:
sentence_int = torch.tensor( [dc[s] for s in sentence.replace(',', '').split()])
print(sentence_int) # Tokenized sequence

tensor([6, 3, 0, 4, 5, 2, 1])


In [34]:
len(dc)  # Vocabulary size

7

In [35]:
vocab_size = len(dc) # 768 - 4096
emb = torch.nn.Embedding(vocab_size, 3) # Zufallsinitialisierte Embedding-Matrix
emb.weight.data # token 0 

tensor([[ 0.9305,  0.1214, -0.2999],
        [ 1.7034,  0.3775,  0.9900],
        [ 0.6404, -1.2503,  1.0156],
        [ 1.9391, -0.7215,  0.7416],
        [-0.6201,  0.1858, -0.1213],
        [ 0.0224,  0.3940,  0.8843],
        [ 0.4107, -0.6532,  1.0826]])

In [36]:
sentence_emb = emb(sentence_int).detach() # Embedding-Vektoren für den Satz
sentence_emb

tensor([[ 0.4107, -0.6532,  1.0826],
        [ 1.9391, -0.7215,  0.7416],
        [ 0.9305,  0.1214, -0.2999],
        [-0.6201,  0.1858, -0.1213],
        [ 0.0224,  0.3940,  0.8843],
        [ 0.6404, -1.2503,  1.0156],
        [ 1.7034,  0.3775,  0.9900]])

In [37]:
# Q, K , V

d = sentence_emb.shape[1]  
d_Q, d_K, d_V = 2, 2, 4 # Dimensionen für Q, K, V

W_Q_L = torch.nn.Linear(d_Q, d, bias=False) 
# oder 
W_Q = torch.nn.Parameter(torch.rand(d, d_Q)) 
#W_Q.weight.data
W_K = torch.nn.Parameter(torch.rand(d, d_K))
W_V = torch.nn.Parameter(torch.rand(d, d_V))

W_K


Parameter containing:
tensor([[0.3456, 0.8993],
        [0.1661, 0.7828],
        [0.3031, 0.3803]], requires_grad=True)

In [38]:
x_0 = sentence_emb[0] # Erster Token aus Embedding-Satz
q_0 = x_0 @ W_Q # drei Repräsentationen    
k_0 = x_0 @ W_K   
v_0 = x_0 @ W_V   
q_0, k_0, v_0

(tensor([0.1630, 0.2545], grad_fn=<SqueezeBackward4>),
 tensor([0.3616, 0.2698], grad_fn=<SqueezeBackward4>),
 tensor([0.4538, 0.3669, 0.6983, 0.5449], grad_fn=<SqueezeBackward4>))

In [39]:
sentence_emb

tensor([[ 0.4107, -0.6532,  1.0826],
        [ 1.9391, -0.7215,  0.7416],
        [ 0.9305,  0.1214, -0.2999],
        [-0.6201,  0.1858, -0.1213],
        [ 0.0224,  0.3940,  0.8843],
        [ 0.6404, -1.2503,  1.0156],
        [ 1.7034,  0.3775,  0.9900]])

In [40]:
keys = sentence_emb @ W_K  # Alle Keys
values = sentence_emb @ W_V  # Alle Values
queries = sentence_emb @ W_Q  # Alle Queries

scores_raw = queries @ keys.T # QK^T für alle Tokens / Relevanzmaß


In [41]:
import torch.nn.functional as F

scores = scores_raw / math.sqrt(d_K)  # Skalierung
attn_weights = F.softmax(scores, dim=1)  # Softmax über Zeilen
attn_weights  # Aufmerksamkeitsgewichte

tensor([[0.1292, 0.1679, 0.1407, 0.1060, 0.1384, 0.1221, 0.1958],
        [0.0621, 0.2169, 0.0995, 0.0258, 0.0890, 0.0469, 0.4598],
        [0.1035, 0.1953, 0.1328, 0.0670, 0.1249, 0.0896, 0.2869],
        [0.1570, 0.1058, 0.1351, 0.2069, 0.1401, 0.1716, 0.0834],
        [0.1284, 0.1678, 0.1414, 0.1059, 0.1384, 0.1211, 0.1969],
        [0.1308, 0.1658, 0.1408, 0.1089, 0.1389, 0.1245, 0.1903],
        [0.0507, 0.2152, 0.0879, 0.0185, 0.0771, 0.0366, 0.5140]],
       grad_fn=<SoftmaxBackward0>)

In [42]:
# Attention(Q,K,V) = softmax(QK^T/sqrt(d_K))V
attn_output = attn_weights @ values  # Gewichtetet Summe der Values
attn_output  # Kontextvektor 

tensor([[0.3600, 0.4638, 1.0553, 1.0197],
        [0.4899, 0.7595, 1.6412, 1.6270],
        [0.4083, 0.5723, 1.2835, 1.2578],
        [0.2786, 0.3039, 0.6773, 0.6259],
        [0.3600, 0.4651, 1.0572, 1.0219],
        [0.3569, 0.4569, 1.0404, 1.0041],
        [0.5130, 0.8155, 1.7372, 1.7254]], grad_fn=<MmBackward0>)

In [43]:
# (1,2,1) -> 1
# (1,2,0) -> 0
# (4,2,1) -> 1
# (5,2,0) -> 0
# (6,2,1) -> 1
# (7,2,0) -> 0
# (1,2,1) -> 1
# (1,2,0) -> 0
# (1,2,1) -> 1
# (1,2,0) -> 0

In [44]:
block_size = attn_weights.shape[0]
mask = torch.tril(torch.ones(block_size, block_size))
print(mask)
attn_weights*mask

tensor([[1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1.]])


tensor([[0.1292, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0621, 0.2169, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1035, 0.1953, 0.1328, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1570, 0.1058, 0.1351, 0.2069, 0.0000, 0.0000, 0.0000],
        [0.1284, 0.1678, 0.1414, 0.1059, 0.1384, 0.0000, 0.0000],
        [0.1308, 0.1658, 0.1408, 0.1089, 0.1389, 0.1245, 0.0000],
        [0.0507, 0.2152, 0.0879, 0.0185, 0.0771, 0.0366, 0.5140]],
       grad_fn=<MulBackward0>)

In [45]:
row_sums = mask.sum(dim=1, keepdim= True)
mask_norm = mask / row_sums
mask_norm

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429]])

In [ ]:
mask = torch.tril(torch.ones(block_size, block_size), diagonal=-1)
scores_masked = scores.masked_fill(mask.T.bool(), -torch.inf)
attn_weights_masked = torch.softmax(scores_masked, dim=1)
attn_weights_masked

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2226, 0.7774, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2398, 0.4525, 0.3077, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2596, 0.1749, 0.2234, 0.3421, 0.0000, 0.0000, 0.0000],
        [0.1883, 0.2461, 0.2074, 0.1553, 0.2029, 0.0000, 0.0000],
        [0.1616, 0.2048, 0.1739, 0.1345, 0.1716, 0.1537, 0.0000],
        [0.0507, 0.2152, 0.0879, 0.0185, 0.0771, 0.0366, 0.5140]],
       grad_fn=<SoftmaxBackward0>)